In [ ]:
import numpy as np

import matplotlib.pyplot as plt

import torch

import sys
sys.path.insert(0, './../')
import sdss_psf
import simulated_datasets_lib
import starnet_lib
import inv_KL_objective_lib as objectives_lib

import json

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print('torch version: ', torch.__version__)

In [ ]:
# load PSF
psf_fit_file = '../../celeste_net/sdss_stage_dir/3900/6/269/psField-003900-6-0269.fit'
print('psf file: \n', psf_fit_file)

In [ ]:
np.random.seed(43534)
_ = torch.manual_seed(24534)

# Draw data

In [ ]:
# data parameters
with open('../data/default_star_parameters.json', 'r') as fp:
    data_params = json.load(fp)

data_params['slen'] = 101
data_params['min_stars'] = 2000
data_params['max_stars'] = 2000
data_params['alpha'] = 0.5

print(data_params)


In [ ]:
max_stars = data_params['max_stars']

In [ ]:
n_images = 1

star_dataset = \
    simulated_datasets_lib.load_dataset_from_params(psf_fit_file,
                            data_params,
                            n_images = n_images,
                            add_noise = False)

num_unlabeled = star_dataset.fluxes.shape[0]
print('num unlabeled', num_unlabeled)

In [ ]:
# get loader 
batchsize = n_images

loader = torch.utils.data.DataLoader(
                 dataset=star_dataset,
                 batch_size=batchsize,
                 shuffle=False)

loader.dataset.set_params_and_images()

In [ ]:
for _, data in enumerate(loader):
    true_fluxes = data['fluxes']
    true_locs = data['locs']
    images = data['image']
    backgrounds = data['background']

In [ ]:
images.shape

In [ ]:
true_locs.shape

In [ ]:
i = np.random.choice(batchsize, 1)[0]

In [ ]:
for i in range(1): 
    # observed image 
    plt.matshow(images[i, 0, :, :])

    # plot locations 
#     locs_i = true_locs[i]
#     n_stars_i = true_n_stars[i]
#     fluxes_i = true_fluxes[i]
#     locs_y = (locs_i[0:int(n_stars_i), 0]) * (images.shape[-1] - 1) 
#     locs_x = (locs_i[0:int(n_stars_i), 1]) * (images.shape[-1] - 1)

#     plt.scatter(x = locs_x, y = locs_y, c = 'b')
    
#     plt.title('Observed image; nstars {}\n'.format(n_stars_i))


In [ ]:
plt.hist(torch.log10(true_fluxes.flatten()))

# Pass through VAE

In [ ]:
star_encoder = starnet_lib.StarEncoder(full_slen = data_params['slen'], 
                                        stamp_slen = 9,
                                        step = 2,
                                        edge_padding = 3, 
                                        n_bands = 1, 
                                        max_detections = 20)

### Check my extraction of subimages

In [ ]:
image_stamps, subimage_locs, subimage_fluxes, n_stars, is_on_array = \
    star_encoder.get_image_stamps(images, true_locs, true_fluxes, trim_images=False)

In [ ]:
plt.hist(n_stars)

In [ ]:
is_on_array2 = simulated_datasets_lib.get_is_on_from_n_stars(n_stars, max(n_stars))

In [ ]:
assert torch.all(is_on_array2 == is_on_array)

In [ ]:
is_on_array.shape

In [ ]:
subimage_locs

### this is what the NN sees

In [ ]:
import plotting_utils

In [ ]:
for i in range(10): 
    f, axarr = plt.subplots(1, 3, figsize=(16, 6))
    indx = int(np.random.choice(image_stamps.shape[0], 1))
    
    which_nonzero = is_on_array[indx].type(torch.bool)
    
    im1 = axarr[0].matshow(torch.log(image_stamps[indx].squeeze()))
    patch_slen = star_encoder.stamp_slen - 2 * star_encoder.edge_padding
    axarr[0].scatter(subimage_locs[indx, which_nonzero, 1] * (patch_slen - 1) + star_encoder.edge_padding, 
                    subimage_locs[indx, which_nonzero, 0] * (patch_slen - 1) + star_encoder.edge_padding, 
                    color = 'b')
    
    plotting_utils.plot_subimage(axarr[1], torch.log(images.squeeze()), None, 
                                true_locs.squeeze() * (images.shape[-1] - 1), 
                                int(star_encoder.tile_coords[indx, 0]), 
                                int(star_encoder.tile_coords[indx, 1]), 
                                 subimage_slen = star_encoder.stamp_slen)
    
    axarr[1].axvline(x=star_encoder.edge_padding, color = 'r')
    axarr[1].axvline(x=star_encoder.stamp_slen - star_encoder.edge_padding - 1, color = 'r')
    axarr[1].axhline(y=star_encoder.edge_padding, color = 'r')
    axarr[1].axhline(y=star_encoder.stamp_slen - star_encoder.edge_padding - 1, color = 'r')


### Check my parameters

In [ ]:
background_stamps = backgrounds.mean() # TODO

In [ ]:
image_stamps, subimage_locs, subimage_fluxes, n_stars, is_on_array = \
    star_encoder.get_image_stamps(images, true_locs, true_fluxes, trim_images=False)

In [ ]:
plt.hist(n_stars)

In [ ]:
logit_loc_mean, logit_loc_log_var, \
    log_flux_mean, log_flux_log_var, log_probs = \
        star_encoder(image_stamps, background_stamps, n_stars)

In [ ]:
logit_loc_mean.shape

In [ ]:
logit_loc_mean

In [ ]:
locs_log_probs_all = \
        objectives_lib.get_locs_logprob_all_combs(subimage_locs,
                                    logit_loc_mean,
                                    logit_loc_log_var)


In [ ]:
locs_log_probs_all.shape

In [ ]:
flux_log_probs_all = \
        objectives_lib.get_fluxes_logprob_all_combs(subimage_fluxes, \
                                    log_flux_mean, log_flux_log_var)

In [ ]:
flux_log_probs_all.shape

In [ ]:
is_on_array.shape

In [ ]:
import hungarian_alg

In [ ]:
locs_log_probs_all.shape

In [ ]:
is_on_array.shape

In [ ]:
# is_on_array[is_on_array.float().sum(dim = 1) > star_encoder.max_detections] = 0
perm = objectives_lib.run_batch_hungarian_alg_parallel(locs_log_probs_all, is_on_array.type(torch.bool)).to(device)

In [ ]:
is_on_array.shape

In [ ]:
locs_log_probs_all.shape

In [ ]:
perm.shape

In [ ]:
locs_log_probs_all.shape

In [ ]:
objectives_lib._permute_losses_mat(locs_log_probs_all, perm).shape

In [ ]:
objectives_lib._permute_losses_mat(locs_log_probs_all, perm).shape

In [ ]:
objectives_lib.get_encoder_loss(star_encoder, 
                                images, 
                                backgrounds, 
                               true_locs, 
                               true_fluxes)

In [ ]:
objectives_lib.eval_star_encoder_loss(star_encoder, loader)

In [ ]:
plt.hist(n_stars)

In [ ]:
len(loader)

In [ ]:
image = image_stamps; background = 686.

In [ ]:
log_img = torch.log(image - background + 1000)

# means = log_img.view(image.shape[0], self.n_bands, -1).mean(-1)
# stds = log_img.view(image.shape[0], self.n_bands, -1).std(-1)
mins = log_img.view(image.shape[0], 1, -1).min(-1)[0]
maxes = log_img.view(image.shape[0], 1, -1).max(-1)[0]

In [ ]:
mins[0]

In [ ]:
maxes[0]

In [ ]:
foo = (log_img - mins.unsqueeze(-1).unsqueeze(-1)) / (maxes - mins).unsqueeze(-1).unsqueeze(-1)

In [ ]:
torch.min(foo[0])

In [ ]:
torch.max(foo[0])

In [ ]:
foo = np.loadtxt('../fits/test_losses')

In [ ]:
foo[:, 6]